# The analysis of multiscale model results & running TDSE interactively

This notebook shows various analyses of the results of the multiscale model. First, we investigate the pulse shaping due to the non-linear propagation, the plasma channel. Then we analyse the build-up of the XUV signal and its structure. Next, we will get more insight directly into the generating process inside the medium by studying the spectra. To conclude the analysis, we outline possibilities to fasten the analysis. Finally, we show the TDSE solver applied to a custom electric field defined directly in this notebook. We use various tools to analyse the result.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## Load data

We load the data from the pulse propagation in the Pythonic data container. It contains the data about the pulse propagation and some firther characteristics. The data from the micrscopic response and harmonic signal will be loaded later (these data are large, and we will need only a part of them according to the chosen analyses.)

In [ ]:

from contextlib import ExitStack

series_version = 'v3'

h5files = [os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'Bessel', series_version, foo) for foo in 
            ['results_Bessel_1.h5', 'results_Bessel_7.h5', 'results_Bessel_3.h5', 'results_Bessel_9.h5']]

# simulation = 'bigpf'
# h5file1 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results.h5')
# h5file2 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results_Hankel.h5')

# with h5py.File(h5file1,'r') as f, h5py.File(h5file2,'r') as f2:

#     # load cuprad data = pulse propagation
#     CUPRAD_res = dfC.get_data(f)
#     CUPRAD_res2 = dfC.get_data(f2)
#     # CUPRAD_res.get_plasma(f)

CUPRAD_res = []
with ExitStack() as stack:
    # Open all files and store file objects in a list
    files = [stack.enter_context(h5py.File(h5file, 'r')) for h5file in h5files]
    
    # Load data from each file and append it to CUPRAD_res
    for f in files:
        CUPRAD_res.append(dfC.get_data(f))


## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Then we plot the plasma channel create by the passage of the pulse. We show both absolute density of free electrons and also relative to the local density.

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Possible density modulation is relative to this reference pressure, whcih is the reason we use the average pressure in our examples. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pyrhonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data or just by sychronising the clocks in the $t$-grid).

In [ ]:

tlim = np.asarray([t_plot_span*CUPRAD_res[0].pulse_duration_entry for t_plot_span in (-2. , 2.)])
# rlim = 600

print([foo.Nz for foo in CUPRAD_res])

In [ ]:
# Code to generate the animated figure

k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res[0].tgrid,1.05*tlim)
# k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

# fig = plt.figure(figsize=(14, 5.5))


fig, axes = plt.subplots(2, 2, figsize=(15, 9))  # Create a 2x2 grid of subplots
axes = axes.flatten()

# pc1 = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], 1e6*CUPRAD_res.rgrid, CUPRAD_res.E_zrt[0,:,k_t_min:k_t_max], shading='auto', cmap = 'seismic')
# pc2 = ax2.pcolormesh(1e15*CUPRAD_res2.tgrid[k_t_min:k_t_max], 1e6*CUPRAD_res2.rgrid, CUPRAD_res2.E_zrt[0,:,k_t_min:k_t_max], shading='auto', cmap = 'seismic')

pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min:k_t_max],
                       1e6*CUPRAD_res[k1].rgrid,
                       CUPRAD_res[k1].E_zrt[0, :, k_t_min:k_t_max],
                       shading='auto', cmap='seismic')
    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbars.append(cbar)

    # Set axis properties
    ax.set_xlim(tlim)
    ax.set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[0]))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')

axes[0].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')  # Set ylabel only for the first plot

# Create colorbars using list comprehensions


# ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
# ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
# ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# cbar1 = fig.colorbar(pc1, ax=ax1)
# cbar2 = fig.colorbar(pc2, ax=ax2)
# # cbar.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)




def update(frame):
    for k1 in range(len(CUPRAD_res)):
        # Update the data for each subplot
        data = CUPRAD_res[k1].E_zrt[8*frame, :, k_t_min:k_t_max]
        
        # Update the colors
        pcs[k1].set_array(data.ravel())
        max_value_symmetric = np.max(np.abs((data.min(), data.max())))
        pcs[k1].set_clim(-max_value_symmetric,max_value_symmetric)
        cbars[k1].update_normal(pcs[k1])

        axes[k1].set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[8*frame]))

    return pcs

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res[0].zgrid)//8, blit=True)

HTML(ani.to_jshtml())


In [ ]:
# Code to generate the figure of plasma channel


fig = plt.figure(figsize=(14, 5.5))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right


symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res.plasma.rgrid,CUPRAD_res.plasma.value_zrt[:,:,-1])
pc1 = ax1.pcolormesh(1e3*CUPRAD_res.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, shading = 'auto')
ax1.set_ylim(-rlim,rlim)
cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')


# local_density_modulation = CUPRAD_res.effective_neutral_particle_density *\
#                                 interpolate.interp1d(
#                                 CUPRAD_res.density_mod_zgrid,
#                                 CUPRAD_res.density_mod_profile_relative,
#                                 bounds_error = False,
#                                 fill_value = (CUPRAD_res.density_mod_profile_relative[0],
#                                             CUPRAD_res.density_mod_profile_relative[-1]),
#                                 copy = False
#                                 )(CUPRAD_res.plasma.zgrid)

# pc2 = ax2.pcolormesh(1e3*CUPRAD_res.plasma.zgrid,
#                1e6*symmetric_y,
#                1e2*(mn.symmetrize_y(CUPRAD_res.plasma.rgrid,
#                         CUPRAD_res.plasma.value_zrt[:,:,-1] /
#                             np.outer(np.ones(len(CUPRAD_res.plasma.rgrid)),
#                                         local_density_modulation).T
#                         )[1]).T,
#                 shading = 'auto')
# ax2.set_ylim(-rlim,rlim)

# cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')

ax1.set_ylabel(r'$\rho~[\mu \mathrm{m}]$')
ax1.set_xlabel(r'$z~[\mathrm{mm}]$')
ax2.set_xlabel(r'$z~[\mathrm{mm}]$')

cbar1.ax.set_xlabel('plasma density $[\mathrm{m}^{-3}]$')
cbar2.ax.set_xlabel('relative plasma density [%]')

plt.show()
